In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('kmdocs').getOrCreate()

In [0]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.ml.clustering import KMeans

In [0]:
df = spark.read.format("libsvm").load("/FileStore/tables/sample_kmeans_data.txt")
df.show() 

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [0]:
fdf = df.select('features')
fdf.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [0]:
km = KMeans().setK(3).setSeed(1)

In [0]:
model = km.fit(fdf)

In [0]:
wssse = model.summary.trainingCost
wssse

Out[24]: 0.07499999999994544

In [0]:
fdf.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [0]:
model.clusterCenters()

Out[26]: [array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [0]:
res = model.transform(fdf)

In [0]:
res.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 2|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+